### By: Ricardo Montalvo Guzman

### NOTE: DO NOT FORGET TO RESTART THE KERNEL EACH TIME A GAME IS OVER

### You may choose MTD-fthe algorithm to use heuristic_score1 or heuristic_score2 two cells below.


In [1]:
import copy

class Board:
    
    frontier = {}
    
    def __init__(self, other = None):
        self.player = 'x'
        self.opponent = 'o'
        self.empty = '.'
        self.cols = 7
        self.rows = 6
        self.state = [[self.empty for i in range(self.cols)] for i in range(self.rows)]
        if other:
            self.__dict__ = copy.deepcopy(other.__dict__)

            
    # The method move(self, x) updates the state of the board after the current player
    # puts a token in column x. Then the turns are swapped.
    def move(self, x):
        #x is the col were to put a token
        board = Board(self)
        for y in range(board.rows): 
            if board.state[self.rows - y - 1][x] == board.empty:
                board.state[self.rows - y - 1][x] = board.player
                break
        board.player,board.opponent = board.opponent,board.player
        return board
    
    
    # The method heuristic(self) computes the difference of the heuristic_score
    # of player and opponent.
    def heuristic(self):
        return self.heuristic_score(self.player) - self.heuristic_score(self.opponent)
    
    
    # The method potential_lines(self, player) returns all the 4 length lines that player 
    # can potentially control at a point before the game is won.
    def potential_lines(self, player):

        lines = []
        
        # We check the vertical lines.
        for y in range(0,3):
            for x in range(0,7):
                four_vertical = [self.state[y + a][x] for a in range(4)]
                if list(set(four_vertical) - set([player, self.empty])) == []:
                    lines.append([(x, y + a) for a in range(4)])

        # We check the horizontal lines.            
        for y in range(0,6):
            for x in range(0,4):
                four_horizontal = [self.state[y][x + a] for a in range(4)]
                if list(set(four_horizontal) - set([player, self.empty])) == []:
                    lines.append([(x + a, y) for a in range(4)])

        # We check the positive-sloped diagonal lines            
        for y in range(3,6):
            for x in range(0,4):
                four_pos_diagonal = [self.state[y - a][x + a] for a in range(4)]
                if list(set(four_pos_diagonal) - set([player, self.empty])) == []:
                    lines.append([(x + a, y - a) for a in range(4)])
        
        # We check the negative-sloped diagonal lines            
        for y in range(3,6):
            for x in range(3,7):
                four_neg_diagonal = [self.state[y - a][x - a] for a in range(4)]
                if list(set(four_neg_diagonal) - set([player, self.empty])) == []:
                    lines.append([(x - a, y - a) for a in range(4)])
        
        return lines
    
    # The method iterative_deepening(self) returns the best identified move using the
    # MDT(f) algorithm to a depth of 5, as well as its heuristic score
    def iterative_deepening(self):
        g = (3, None)
        for d in range(1, 5):
            g = self.mtdf(g, d)
        return g
    
    # Here we retrieve the best moves available with the null window search
    def mtdf(self, g, d):
        upperBound = 10000
        lowerBound = -10000
        choice = g
        while lowerBound < upperBound:
            if g[0] == lowerBound:
                beta = g[0] + 1
            else:
                beta = g[0]
            g = self.minimax(True, d, beta - 1, beta)
            if g[1] != None:
                choice = g
            if g[0] < beta:
                upperBound = g[0]
            else:
                lowerBound = g[0]
        return choice
    
    # minimax() is implemented with memorized alpha-beta prunning
    # therefore it seeks if the nodes are in the frontier first
    # a series of boolean test are conducted before implementing 
    # minimax recursively, after which the frontier is expanded and 
    # updated
    def minimax(self, player, depth, alpha, beta):
        lower = Board.frontier.get(str(self)+str(depth)+'lower',None)
        upper = Board.frontier.get(str(self)+str(depth)+'upper',None)
        if lower != None:
            if lower >= beta:
                return (lower, None)
            alpha = max(alpha,lower)
        if upper != None:
            if upper <= alpha:
                return (upper, None)
            beta = max(beta, upper)
        if self.won():
            if player:
                return (-99999, None)
            else:
                return (99999, None)
        elif self.tied():
            return(0, None)
        elif depth == 0:
            return(self.heuristic(),None)
        elif player:
            choice = (alpha, None)
            for x in range(0,7):
                if self.state[0][x] == self.empty:
                    value = self.move(x).minimax(not player, depth - 1, choice[0], beta)[0]
                    if value > choice[0]:
                        choice = value, x
                    if value > beta:
                        break
        else:
            choice = (beta, None)
            for x in range(0,7):
                if self.state[0][x] == self.empty:
                    value = self.move(x).minimax(not player, depth - 1, alpha, choice[0])[0]
                    if value < choice[0]:
                        choice =  value, x
                    if alpha > value:
                        break
        if choice[0] <= alpha:
            Board.frontier[str(self) + str(depth) + "upper"] = choice[0]
        elif choice[0] >= beta:
            Board.frontier[str(self) + str(depth) + "lower"] = choice[0]
        return choice
    
    # best(self) fetches the 1-indezed element returned by iterative_deepening(),
    # in other words, the best available column choice
    def best(self):
        return self.iterative_deepening()[1] 
    
    # tied() returns True if and only if the game ends in a tie
    def tied(self):
        for x in range(0,7):
            for y in range(0,6):
                if self.state[y][x] == self.empty:
                    return False
        return True
 
    # won() returns the coordinates of a winning line when this occurs
    def won(self):
        
        # The variable player is set to self.opponent because at the time when
        # a winning occurs, the turn is transferred before the machine realizes
        # the game has ended
        player = self.opponent
        lines = self.potential_lines(player)

        for line in lines:
            four = 0
            for point in line:
                if self.state[point[1]][point[0]] == player:
                    four += 1
            if four == 4:
                return line
        return None
    
    def __str__(self):
        string = ''
        for y in range(0,6):
            for x in range(0,7):
                string += ' ' + self.state[y][x]
            string += '\n'
        return string
    
    

# Here the two heuristic scores, in the next cell the programmer can choose which one to
# add to the class Board.
def heuristic_score1(self, player):
    lines = self.potential_lines(player)
    score = 0
    for line in lines:
        pieces = 0
        height = []
        for x,y in line:
            if self.state[y][x] == player:
                pieces = pieces + 1
            elif self.state[y][x] == self.empty:
                height.append(6 - y + 1)
        heightscore = 6 - int(sum(height) / float(len(height)))
        score += pieces*heightscore

    return score

def heuristic_score2(self, player):
    lines = self.potential_lines(player)
    score = 0
    for line in lines:
        pieces = 0
        for x,y in line:
            if self.state[y][x] == player:
                pieces = pieces + 1
        if pieces == 2:
            score += 5
        elif pieces == 3:
            score += 100 

    return score

In [ ]:
### PLUG IN HEURISTIC HERE
Board.heuristic_score = heuristic_score2


from tkinter import Tk, Button, Frame, Canvas

# The program asks the user whether they want to be the first player or not
while True:
    first_player = input("Would you like to be the first player? (Answer must be yes or no)")
    if not first_player in ['yes', 'no']:
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

class GUI:
    
    def __init__(self):
        self.app = Tk()
        self.app.title('Connect Four')
        # The following lines reflect whether the user wants to be the first player or not
        if first_player == 'yes':
            self.board = Board()
        elif first_player == 'no':
            self.board = Board().move(3)
        self.buttons = {}
        self.frame = Frame(self.app)
        self.tiles = {}

    # Buttons for the human player to select a row
        for x in range(self.board.cols):
            handler = lambda x = x: self.move(x)
            button = Button(self.app, command = handler, text = x + 1)
            button.grid(row = 0, column = x, sticky = "WE")
            self.buttons[x] = button
        self.frame.grid(row = 1, column = 0 , columnspan = self.board.cols)
        
        for y in range(len(self.board.state)):
            for x in range(len(self.board.state[y])):
                tile = Canvas(self.frame, width=60, height=50, bg="purple", highlightthickness=0)
                tile.grid(row=self.board.rows-1-y, column=x + 1)
                self.tiles[x,y] = tile
        self.update()
    
    # move(self, x) expects the user to choose a row and immediately after takes the move of the 
    # AI opponent
    def move(self,x):
        self.app.config(cursor="watch")
        self.app.update()
        self.board = self.board.move(x)
        self.update()
        move = self.board.best()
        if move != None:
            self.board = self.board.move(move)
            self.update()
        self.app.config(cursor="")

    # update(self) updates the displayed board
    def update(self):
        for y in range(len(self.board.state)):
            for x in range(len(self.board.state[y])):
                text = self.board.state[y][x]
                if (text=='.'):
                    self.tiles[x,5 - y].create_oval(10, 5, 50, 45, fill="black", outline="orange", width=1)
                if (text=='x'):
                    self.tiles[x,5 - y].create_oval(10, 5, 50, 45, fill="yellow", outline="orange", width=1)
                if (text=='o'):
                    self.tiles[x,5- y].create_oval(10, 5, 50, 45, fill="red", outline="orange", width=1)

    
        for x in range(self.board.cols):
            if self.board.state[0][x] == self.board.empty:
                self.buttons[x]['state'] = 'normal'
            else:
                self.buttons[x]['state'] = 'disabled'
        winning = self.board.won()
        if winning:
            for x,y in winning:
                self.tiles[x,5 - y].create_oval(25, 20, 35, 30, fill="black")
            for x in range(self.board.cols):
                self.buttons[x]['state'] = 'disabled'


    def mainloop(self):
        self.app.mainloop()

    
if __name__ == '__main__':
    GUI().mainloop()

Would you like to be the first player? (Answer must be yes or no)yes
